# CS 282
### Programming Assignment 1
#### Item 1

Jan Lendl R. Uy

2019-00312

#### Sytem Specifications
- macOS Sequoia 15.0.1
- Macbook Air M1 (ARM), 8-Core CPU and 8-Core GPU

#### Notes when Running the Notebook
- For items 1d and 1e, press any key to terminate OpenCV2 windows.
  - OpenCV2 windows tend to be buggy when attempted to be closed. They are best terminated through the following:
    - For macOS: Click 'Force Quit' in Activity Monitor
    - For Windows: Click 'End Task' in Task Manager

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import cv2
import math

### 1(a)

Select a negative floating-point number. Take its absolute value, round it, and then take its ceiling and floor. Print the results to screen.

In [3]:
# x = -5.44444
try:
    x = float(input("Enter a negative floating-point number: "))
    print(f"Chosen floating-point number: {x}")
except:
    raise ValueError("Inputted value is not a float!")

y = abs(x)
print(f"Absolute value of {x}: {y}")

z = round(y, 2)
print(f"Rounded-off value of {y}: {z}")

# Note: opencv2 has no built-in functions for getting the ceiling and floor
print(f"Ceiling of {z}: {math.ceil(z)}")
print(f"Floor of {z}: {math.floor(z)}")

Chosen floating-point number: -5.1444
Absolute value of -5.1444: 5.1444
Rounded-off value of 5.1444: 5.14
Ceiling of 5.14: 6
Floor of 5.14: 5


### 1(b)

Generate a 3 ×4 matrix whose elements are random integers. Print the matrix.

In [22]:
dst_mtx = cv2.UMat(3, 4, cv2.CV_32SC1)

# Fill it with random values
# Range of possible random values are from 0 to 255
cv2.randu(dst_mtx, 0, 256)

# Retrieve array from GPU for printing
dst_mtx = cv2.UMat.get(dst_mtx)

print(f"Random 3x4 matrix:\n{dst_mtx}")

Random 3x4 matrix:
[[131  63   5 237]
 [220 209 155  88]
 [ 28  12  58  99]]


### 1(c)

Declare matrix variables A,B, and integer variable c within OpenCV. Assign them with the following values:

In [5]:
# Note: opencv2 arrays are a wrapper of numpy arrays
# Thus, the usage of numpy for matrix operations is inevitable
import numpy as np

In [6]:
# Convert list of 3x1 tuples to NumPy array and then to cv2.Mat
# Note: opencv2 requires an input array of type numpy.array 
# for initialization
A_list = [
    [(6, 0, 2), (2, 6, 5), (9, 7, 1)],
    [(1, 6, 4), (5, 4, 9), (6, 9, 2)],
    [(3, 5, 9), (5, 3, 7), (8, 3, 3)]
]

A = cv2.Mat(np.array(A_list, dtype=np.float32))

B_list = [
    [(9, 1, 9), (7, 0, 5), (2, 0, 3)],
    [(9, 2, 3), (4, 3, 6), (0, 2, 9)],
    [(7, 2, 6), (0, 3, 8), (4, 2, 1)]
]

B = cv2.Mat(np.array(B_list, dtype=np.float32))

c = 0.7
d = 0.55

### 1(c)(i)

$cA+ (1−c)B+ d$

In [7]:
answer = cv2.addWeighted(A, c, B, (1-c), d)
print(f"cA + (1-c)B + d =\n{answer}")

cA + (1-c)B + d =
[[[7.45 0.85 4.65]
  [4.05 4.75 5.55]
  [7.45 5.45 2.15]]

 [[3.95 5.35 4.25]
  [5.25 4.25 8.65]
  [4.75 7.45 4.65]]

 [[4.75 4.65 8.65]
  [4.05 3.55 7.85]
  [7.35 3.25 2.95]]]


### 1(c)(ii)

$A^{-1}$

Note: Get the matrix inverse via SVD given the formula $A^{-1} = V_{A} \Sigma^{-1}_{A} U^T_{A}$, where $V$ is a matrix of right singular vectors, $\Sigma$ is a diagonal matrix of singular values, and $U$ is a matrix of left singular vectors of A

In [8]:
def process_matrix_inverse_by_channel(matrix_array):
    """
    Compute and print the inverse of a matrix separately for each channel.
    """
    # Ensure input is a NumPy array
    if not isinstance(matrix_array, np.ndarray):
        matrix_array = np.array(matrix_array, dtype=np.float32)
    
    # Get the shape
    rows, cols, channels = matrix_array.shape
    
    # Process each channel
    results = []
    for c in range(channels):
        # Extract the channel
        channel = matrix_array[:, :, c]
        
        # Perform SVD on this channel
        s, u, vt = cv2.SVDecomp(channel)
        
        # Calculate inverse for this channel
        s_reciprocal = 1/s.flatten()
        S_inv = np.diag(s_reciprocal)
        channel_inv = vt.T @ S_inv @ u.T
        
        results.append(channel_inv)
        
    return results

In [9]:
A_inverse = process_matrix_inverse_by_channel(A_list)
print("A_inverse:\n")
for i, channel_result in enumerate(A_inverse):
    print(f"Channel {i}:\n{channel_result}")

A_inverse:

Channel 0:
[[-1.0000005 -2.900001   3.3000014]
 [-1.0000002 -2.1000006  2.7000008]
 [ 1.0000002  2.4000006 -2.800001 ]]
Channel 1:
[[-0.10135134  0.0202703   0.17567565]
 [ 0.18243243 -0.23648646  0.28378376]
 [-0.01351352  0.20270269 -0.24324326]]
Channel 2:
[[ 4.3333316e+00 -2.6666656e+00  3.3333322e-01]
 [ 1.9999992e+00 -9.9999946e-01 -6.2145446e-08]
 [-1.7666660e+01  1.0333329e+01 -6.6666621e-01]]


### 1(c)(iii)

$eig(B)$

In [10]:
def process_matrix_eigenvalues_by_channel(matrix_array):
    """
    Compute eigenvalues of a matrix with channels, returning a NumPy array.
    """
    # Ensure input is a NumPy array
    if not isinstance(matrix_array, np.ndarray):
        matrix_array = np.array(matrix_array, dtype=np.float32)

    # Get the shape
    rows, cols, channels = matrix_array.shape
    
    if rows != cols:
        raise ValueError("Matrix must be square!")
    
    # For a row×col matrix, we'll have min(row,col) eigenvalues
    num_eigenvalues = min(rows, cols)
        
    # Process each channel
    results = []
    for c in range(channels):
        # Extract the channel
        channel = matrix_array[:, :, c]
        
        # Compute eigenvalues
        _, eigenvalues, _ = cv2.eigen(channel)
        
        # Store the eigenvalues in the results array
        results.append(eigenvalues)
    
    return results

In [11]:
eig_B = process_matrix_eigenvalues_by_channel(B)
print("eig(B):\n")
for i, channel_result in enumerate(eig_B):
    print(f"Channel {i}:\n{channel_result}")

eig(B):

Channel 0:
[[14.197403 ]
 [ 3.9999998]
 [-1.1974022]]
Channel 1:
[[4.561553  ]
 [1.        ]
 [0.43844712]]
Channel 2:
[[17.032358 ]
 [ 4.8394666]
 [-5.8718247]]


Operation above can be verified by directly obtaining the singular values through SVD as follows.

### 1(c)(iv)

Solve the matrix equation $Ax=b$

In [12]:
def solve_matrix_equation_by_channel(A_array, b_array):
    """
    Solve Ax = b for matrices with channels.
    """

    # Get the shapes
    rows, cols, channels = A_array.shape
        
    # Process each channel
    results = []
    for c in range(channels):
        # Extract the channel
        A_channel = A_array[:, :, c]
        b_channel = b_array[:, c]
        
        # Solve for this channel
        x_channel = cv2.solve(A_channel, b_channel.reshape(-1, 1))[1]
        
        # Store the result in the corresponding channel
        results.append(x_channel)
    
    return results

In [13]:
b_list = [
    (3, 5, 5),
    (7, 2, 8),
    (4, 1, 6)
]

b = cv2.Mat(np.array(b_list, dtype=np.float32))

x = solve_matrix_equation_by_channel(A, b)
print("x:\n")
for i, channel_result in enumerate(x):
    print(f"Channel {i}:\n{channel_result}")

x:

Channel 0:
[[-10.1]
 [ -6.9]
 [  8.6]]
Channel 1:
[[-0.29054055]
 [ 0.722973  ]
 [ 0.0945946 ]]
Channel 2:
[[ 2.3333333]
 [ 2.       ]
 [-9.666667 ]]


### 1(d)

Create a 2D matrix with three channels of type byte and size 100-by-100 and set all values to zero. Draw a red rectangle between with corners (30,10) and (60,40). Display this image.

In [14]:
# Initialize 2D tensor (3D matrix) with zeros
# Use cv2.CV_8UC3 for byte-type with 3 channels
img = cv2.UMat(100, 100, cv2.CV_8UC3)

# Fill with zeros
cv2.rectangle(img, (0, 0), (100, 100), (0, 0, 0), -1)

# Draw red rectangle
# Red channel is the last channel in BGR format
# Set red channel to 255 in rectangle area
# -1 means fill entire shape
cv2.rectangle(img, (30, 10), (60, 40), (0, 0, 255), -1)

# Display image
print("Displaying red rectangle")
cv2.imshow("Red Rectangle", img)
cv2.waitKey(0)

# Clean-up
cv2.destroyAllWindows()

Displaying red rectangle


2025-03-03 21:06:19.488 Python[69123:22071613] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-03 21:06:19.488 Python[69123:22071613] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


### 1(e)

Copy the red channel in (d) above into another matrix for display as grayscale image.

In [16]:
# Only get red channel and discard green and blue channels
img = img.get()
red_channel = img[:, :, 2].copy()

# Display the red channel as grayscale
print("Displaying red rectangle in grayscale")
cv2.imshow("Red Channel as Grayscale", red_channel)
cv2.waitKey(0)

# Clean-up
cv2.destroyAllWindows()

Displaying red rectangle in grayscale
